In [ ]:
#libs for reading data
import xarray as xr
import gcsfs
import glob
import numpy as np
import matplotlib.pyplot as plt
from xhistogram.xarray import histogram

#lib for dask gateway
from dask_gateway import Gateway
from dask.distributed import Client

In [ ]:
gateway = Gateway()
cluster = gateway.new_cluster()
cluster.adapt(minimum=1, maximum=75)
client = Client(cluster)
cluster

In [ ]:
zstore = 'gs://pangeo-nasa-ccmp/zarr'

fs = gcsfs.GCSFileSystem(project='pangeo-181919',requester_pays=True)

ds = xr.open_zarr(fs.get_mapper(zstore))

ds['wspd']=np.sqrt(ds.uwnd**2+ds.vwnd**2)

ds

In [ ]:
high_val = ds.wspd.where(ds.wspd>25)
high_cnt = high_val/high_val
high_lat = high_cnt.sum({'longitude'}).load()

In [ ]:
high_lat.transpose().plot(cmap='YlOrBr',vmin=0,vmax=10)
plt.savefig('../../figures/num_storms.png')